In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import sklearn
import keras.backend as K
from keras.models import Sequential
from keras.layers import Dense
from sklearn.ensemble import RandomForestRegressor
from sklearn import linear_model
from sklearn import svm
import pickle

Using TensorFlow backend.


In [2]:
from notebook.services.config import ConfigManager
ConfigManager().update('notebook', {'ExecuteTime': {
   	'display_absolute_timestamps': False,
    'relative_timing_update_period': 5,
    'template': {
    	'executed': 'started ${start_time}, finished in ${duration}',
    }
}})

{'Cell': {'cm_config': {'lineNumbers': True}},
 'ExecuteTime': {'display_absolute_timestamps': False,
  'relative_timing_update_period': 5,
  'template': {'executed': 'started ${start_time}, finished in ${duration}'}}}

## Finding composition with Enrichment and Burnup
Testing time for various methods

In [4]:
sample_num = 100
enr = np.random.uniform(low=3, high=5, size=(sample_num,))
br = np.random.uniform(low=30000, high=50000, size=(sample_num,))
enr_br = np.column_stack((enr,br))
print(np.shape(enr_br))
enr_br

(100, 2)


array([[3.71414642e+00, 4.46847624e+04],
       [3.80848825e+00, 3.52204432e+04],
       [4.19756912e+00, 4.56150110e+04],
       [4.57009368e+00, 4.53049910e+04],
       [3.87427401e+00, 4.27540918e+04],
       [4.40867569e+00, 4.68354991e+04],
       [3.76042461e+00, 4.06559133e+04],
       [4.08202179e+00, 3.45826817e+04],
       [4.24981721e+00, 3.58600720e+04],
       [4.31736847e+00, 3.70631961e+04],
       [4.92434740e+00, 4.94746335e+04],
       [4.63418723e+00, 3.65371547e+04],
       [3.45642378e+00, 3.58604304e+04],
       [3.06403977e+00, 4.82604315e+04],
       [3.04625232e+00, 3.49570915e+04],
       [4.33135718e+00, 3.92678453e+04],
       [4.82271271e+00, 3.68952337e+04],
       [3.19005822e+00, 4.31689634e+04],
       [3.73834292e+00, 4.21071135e+04],
       [4.58169915e+00, 3.84261397e+04],
       [4.38507282e+00, 4.10175458e+04],
       [3.91940731e+00, 4.10071503e+04],
       [4.32854868e+00, 3.87957637e+04],
       [3.20775200e+00, 4.58071175e+04],
       [3.659192

## Pickle and model

In [32]:
# load pickled dictionary with
# key: isotope
# value: trained model
dep_dict = pickle.load(open('lin_dep.pkl',"rb"))

In [42]:
# get time for rom depletion for one assembly
# given enrichment and burnup value,
# predict UNF composition

def get_a(f):
    z = ''
    for i in f:
        if i.isdigit():
            z += i
    return int(z)
def deplete_using_rom(dep_dict, enr_br):
    depleted_composition_rom = {}
    for indx, enr_br_set in enumerate(enr_br):
        depleted_composition_rom[indx] = {}
        x = [enr_br_set]
        for iso, alg in dep_dict.items():
            if 'alg' in iso:
                continue
            else:
                if any(i.isdigit() for i in dep_dict[iso + '_alg']):
                    deg = get_a(dep_dict[iso + '_alg'])
                    poly = sklearn.preprocessing.PolynomialFeatures(degree=deg)
                    depleted_composition_rom[indx][iso] = dep_dict[iso].predict(poly.fit_transform(x))[0]
                else:
                    depleted_composition_rom[indx][iso] = dep_dict[iso].predict(x)[0]

    print('\n')
    return depleted_composition_rom

%time deplete_using_rom(dep_dict, enr_br)




CPU times: user 11.7 s, sys: 0 ns, total: 11.7 s
Wall time: 11.8 s


{0: {'zr-95': 1.4500945347393848e-06,
  'zr-93': 0.0008651161428748424,
  'ra-226': 7.324069647242976e-14,
  'ru-103': 7.066576858840561e-08,
  'ru-106': 7.097650458735987e-05,
  'sb-124': 6.661660005122253e-10,
  'sb-125': 8.784960034829231e-06,
  'pu-238': 0.00019006337584536145,
  'pu-239': 0.0071671908540964155,
  'np-237': 0.0005611840840550434,
  'nb-95': 1.7060354914084005e-06,
  'pa-231': 7.050050714437552e-10,
  'pd-107': 0.0002001807232468263,
  'pm-147': 0.00016094468700842904,
  'pu-240': 0.002153247200246998,
  'pu-241': 0.001524061865916018,
  'pu-244': 1.5791335377904752e-08,
  'pu-242': 0.0003697276349019161,
  'i-129': 0.00015999615697897548,
  'h-3': 5.3475273684558236e-08,
  'kr-85': 2.8347341162255423e-05,
  'nb-94': 1.4229237203409867e-09,
  'cm-248': 1.7491749185241912e-10,
  'cs-134': 8.755445135738424e-05,
  'cs-135': 0.0005890208661891425,
  'cm-245': 1.6560747615551055e-06,
  'cm-244': 2.176586892557184e-05,
  'cm-246': 1.0736043231534429e-07,
  'cm-247': 1.77

## Looking up from db

In [34]:

df = pd.read_csv('./curated.csv', index_col=0)

In [43]:
# get time for `look-up' method for one assembly
# find closest assembly to this burnup and enrichment
def deplete_lookup(df, enr_br):
    depleted_composition_lookup = {}
    for indx, enr_br_set in enumerate(enr_br):
        df['err'] = abs(df['bu'] - enr_br[0][1]) + abs(df['init_enr'] - enr_br[0][0]) * 10000
        minimum_err = df['err'].argmin()
        df = df.drop(columns='err')
        average_assem = df.loc[minimum_err]
        depleted_composition_lookup[indx] = average_assem.iloc[5:].to_dict()
    return depleted_composition_lookup

%time deplete_lookup(df, enr_br)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 4.77 µs


/home/teddy/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: FutureWarning: 'argmin' is deprecated, use 'idxmin' instead. The behavior of 'argmin'
will be corrected to return the positional minimum in the future.
Use 'series.values.argmin' to get the position of the minimum now.
  


CPU times: user 5.41 s, sys: 10.3 s, total: 15.7 s
Wall time: 15.5 s


{0: {'zr-95': 1.1271959671425968e-06,
  'zr-93': 0.0008785766072476435,
  'ra-226': 8.007942832832296e-14,
  'ru-103': 5.6044072223025726e-08,
  'ru-106': 6.720843977030909e-05,
  'sb-124': 5.931242649912018e-10,
  'sb-125': 8.71947895135679e-06,
  'pu-238': 0.0002060786648942572,
  'pu-239': 0.007722372246787458,
  'np-237': 0.0005792439154107984,
  'nb-95': 1.3297389948004582e-06,
  'pa-231': 8.027709245747119e-10,
  'pd-107': 0.0002091611181209754,
  'pm-147': 0.00015431981604866428,
  'pu-240': 0.0022398740743976553,
  'pu-241': 0.001541101816911696,
  'pu-244': 1.865369616801372e-08,
  'pu-242': 0.00038989143257992415,
  'i-129': 0.00016573561969029216,
  'h-3': 5.4848835516391447e-08,
  'kr-85': 3.1885754414854965e-05,
  'nb-94': 1.3604410557573105e-09,
  'cm-248': 6.699713765541278e-11,
  'cs-134': 9.002467578382516e-05,
  'cs-135': 0.0006683309712193964,
  'cm-245': 1.7305922649207755e-06,
  'cm-244': 2.681991699917557e-05,
  'cm-246': 9.87062725452901e-08,
  'cm-247': 1.202930